<a href="https://colab.research.google.com/github/beimnet777/Breast-Cancer-Detection/blob/main/Cancer_Segmentation_with_unet_backbone_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt

In [ ]:
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
!pip install -U -q segmentation-models

In [ ]:
import segmentation_models as sm

In [ ]:
import cv2 as cv

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def resize(image,mask):
  image=tf.keras.layers.MaxPooling2D((2,2))(image)
  mask=tf.keras.layers.MaxPooling2D((2,2))(mask)
  return image,mask
  


In [ ]:

def prepare_dataset(width,height,path):

  for (root ,dirs,files) in os.walk(path):

    X=[]
    y=[]

    for file in files:

      
      image_name=file.split(".")[0]
      if image_name.split("_")[-1]!="mask":

        mask_name=image_name+"_mask.jpg"
        image_path=os.path.join(path,file)
        mask_path=os.path.join(path,mask_name)

        if os.path.exists(mask_path):
          mask_image=cv.imread(mask_path)
          mask_image = np.amax(mask_image, axis=-1)
          # print(mask_image.shape)
          # mask_image = cv.cvtColor(mask_image, cv.COLOR_BGR2GRAY)
          mask_image//=255
          image=cv.imread(image_path)
          image=image/255
          


          mask_image=cv.resize(mask_image, (width, height))
          image=cv.resize(image, (width, height))

          
          # image=np.expand_dims(image,axis=-1)

          X.append(image)
          y.append(mask_image)
    
  X=np.array(X).astype(np.float64)
  y=np.expand_dims(y,axis=-1)
  y=np.array(y).astype(np.float64)
  print(y.shape)
  X,y=resize(X,y)
  return X,y

In [ ]:
X,y=prepare_dataset(512,1024,"/content/drive/MyDrive/Suspicious")


In [ ]:
n_classes=2
activation='sigmoid'

dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
X = preprocess_input1(X)

model1 = sm.Linknet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation,input_shape=(512,256,3))

model1.compile(optimizer="adam", loss=total_loss, metrics=metrics)


print(model1.summary())

In [ ]:
history = model1.fit(X,y, epochs=200,batch_size=8)

In [ ]:
model1.save("second.h5")

In [ ]:
def create_unet(width,height):
  WIDTH=width
  HEIGHT=height

  input=tf.keras.layers.Input((HEIGHT,WIDTH,1))

  
  DROP_OUT=0.2

  c1=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(input)
  d1=tf.keras.layers.Dropout(DROP_OUT)(c1)
  c2=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(d1)
  p1=tf.keras.layers.MaxPooling2D((2,2))(c2)

  c3=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(p1)
  d2=tf.keras.layers.Dropout(DROP_OUT)(c3)
  c4=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(d2)
  p2=tf.keras.layers.MaxPooling2D((2,2))(c4)

  c5=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(p2)
  d3=tf.keras.layers.Dropout(DROP_OUT)(c5)
  c6=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(d3)
  p3=tf.keras.layers.MaxPooling2D((2,2))(c6)

  c7=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(p3)
  d4=tf.keras.layers.Dropout(DROP_OUT)(c7)
  c8=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(d4)
  p4=tf.keras.layers.MaxPooling2D((2,2))(c8)

  c9=tf.keras.layers.Conv2D(256,3,activation="relu",padding="same")(p4)
  d5=tf.keras.layers.Dropout(DROP_OUT)(c9)
  c10=tf.keras.layers.Conv2D(256,3,activation="relu",padding="same")(d5)
  u5=tf.keras.layers.Conv2DTranspose(128,(2,2),strides=(2,2))(c10)
  u5=tf.keras.layers.Concatenate()([u5,c8])



  c11=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(u5)
  d6=tf.keras.layers.Dropout(DROP_OUT)(c11)
  c12=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(d6)
  u6=tf.keras.layers.Conv2DTranspose(64,(2,2),strides=(2,2))(c12)
  u6=tf.keras.layers.Concatenate()([u6,c6])

  c13=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(u6)
  d7=tf.keras.layers.Dropout(DROP_OUT)(c13)
  c14=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(d7)
  u7=tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2))(c14)
  u7=tf.keras.layers.Concatenate()([u7,c4])

  c15=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(u7)
  d8=tf.keras.layers.Dropout(DROP_OUT)(c15)
  c16=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(d8)
  u8=tf.keras.layers.Conv2DTranspose(16,(2,2),strides=(2,2))(c16)
  u8=tf.keras.layers.Concatenate()([u8,c2])

  c17=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(u8)
  d9=tf.keras.layers.Dropout(DROP_OUT)(c17)
  c18=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(d9)


  outputs=tf.keras.layers.Conv1D(1,1,activation="sigmoid")(c18)

  model=tf.keras.Model(inputs=[input],outputs=[outputs])
  model.summary()
  return model

In [ ]:
model=create_unet(256,512)

In [ ]:
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model.compile(optimizer="adam", loss=dice_loss, metrics=metrics)
# metrics

In [ ]:
Prediction=model1.predict(X)

In [ ]:
Prediction[Prediction>=0.5]=1
Prediction[Prediction<0.5]=0

In [ ]:
for index,val in enumerate(Prediction):
  cv2_imshow(np.max(val,axis=-1)*255)
  print("^^^^")
  cv2_imshow(np.array(y[index])*255)
  print("*********************")